In [1]:
import os
import json
import numpy as np
import time
from datetime import datetime , date, timedelta
import sys
import pandas as pd
import numpy as np
import xmltodict
import sys
import numpy as np
from numba import jit
import json
import matplotlib.pyplot as plt
import seaborn as sns
import xmltodict
import numpy.polynomial as p
from multiprocessing import Pool

from project import config
from git_repo.fottech_lib.market_data import dividends
from git_repo.fottech_lib.market_data.dividends import Dividends 
from git_repo.fottech_lib.market_data.dmds import DMDSServices 
from git_repo.fottech_lib import instrumentservice

import project.market_data.dividendscurves as divc
from project.market_data.dividendscurves import DividendsCurves

dmds_date_format = dividends.dmds_date_format
dmds_path_date_format = dividends.dmds_path_date_format

In [24]:
B_to_R = instrumentservice.InstrumentService('prod','APAC')
print(B_to_R.transcode('EURO STOXX 50', target='reuter', partial_match=False))
print(B_to_R.transcode('EURO STOXX 50', target='bloomberg', partial_match=False))


.STOXX50E
SX5E


In [4]:
def get_forward_prices(index, dates1):
    import datetime
    try:
        fetch = datalib.Fetch()
        date_time_obj = datetime.datetime.strptime(dates1[0], '%Y%m%d')
        forward_object = fetch.forward(index, date_time_obj, date_time_obj, 1)
        forward = float(forward_object.to_pandas_series()[0])
        return forward
    except:
        return None

def computing_dividends_universe(universe_indices, date):
    B_to_R = instrumentservice.InstrumentService('prod','APAC')
    dates = []
    dates.append(date)
    dictionary = {}
    for index in universe_indices:
        print('################## Processing dividends for index {} ##################'.format(index))
        try:
            index_ric = B_to_R.transcode(index, target='reuter', partial_match=False)
            index_ric_div = index_ric[1:]
        except:
            index_ric = index
            index_ric_div = index
        print(index_ric)
        print(index_ric_div)
        splits_index = range(1, int(len(dates)/10)+1)
        dates_batch = np.split(dates, splits_index)
        for d in dates_batch:
            dates1 = d.tolist()
            div_schedules = divc.get_dividends_schedules(index_ric_div,dates1)
            forward_prices = []
            try:
                forward_prices.append(get_forward_prices(index_ric, dates1))
            except:
                forward_prices.append(np.nan)
            initial_year = (int)(dates1[0][:4])
            years = range(initial_year, initial_year+12)
            (n,m) = (len(dates1),len(years))
            tot_div = np.zeros((n,m))
            
            for index1,year in enumerate(years):
                year = str(year)
                total_divs = divc.total_div_for_many_days_specific_year_f(div_schedules, forward_prices, index_ric_div, year)
                tot_div[:,index1] = total_divs
        print(forward_prices)
        print(tot_div[0].tolist())
        dictionary[index] = tot_div[0].tolist()
    #divc.save_dict(dictionary, ric, past, future)
    return dictionary

In [ ]:
import datetime
fetch = datalib.Fetch()
date_time_obj = datetime.datetime.strptime('20190710', '%Y%m%d')
forward_object = fetch.forward('.FCHI', date_time_obj, date_time_obj, 1)
forward = float(forward_object.to_pandas_series()[0])
forward

In [5]:
#Loading the indices
file_path = 'data/processed/UNIVERSE/universe_indices.npy'
universe_indices = np.load(file_path)

#Preparing processed data
yesterday = date.today() - timedelta(days=1)
yesterday = yesterday.strftime('%Y%m%d')
dictionnaire = computing_dividends_universe(universe_indices[:3], yesterday)

################## Processing dividends for index CAC 40 ##################
.FCHI
FCHI
number of dates :1
getting dividends from DMDS...
dividends schedules downloaded
time to proceed : 0.30503058433532715 s
[5567.467135325456]
[173.39842800000002, 187.8983174209653, 191.83542487247007, 191.81633027549518, 188.08094900567463, 190.37494087977058, 195.47185467991252, 195.26109777615167, 196.486638996384, 197.1030347037584, 195.03817249260888, 195.55655935757906]
################## Processing dividends for index DAX 30 ##################
.GDAXI
GDAXI
number of dates :1
getting dividends from DMDS...
dividends schedules downloaded
time to proceed : 0.3560357093811035 s
[12373.03887536221]
[399.876156335993, 404.75256796607255, 391.39262499549227, 378.9178738201767, 369.06791267135713, 360.5569953007586, 356.1878113246601, 350.3557477003694, 344.1779180634353, 338.3776197026282, 332.5694677937557, 326.5991053452271]
################## Processing dividends for index FTSE 100 ################

In [ ]:
file_path = '../data/processed/UNIVERSE/Dividends_Spots/spot_schedule_universe_on_2019-06-03_V0.json'
with open(file_path) as json_file:
    data = json.load(json_file)

universe_indices = data['indices']

In [ ]:
file_path = 'data/processed/UNIVERSE/universe_indices.npy'
universe_indices = np.load(file_path)

In [ ]:
universe_indices[-1]

In [ ]:
import datetime
yesterday = date.today() - timedelta(days=1)
yesterday = yesterday.strftime('%Y%m%d')
dictionnaire = computing_dividends_universe([universe_indices[-1]], yesterday)

In [ ]:
file_path = 'data/processed/UNIVERSE/Dividends_Forwards/universe_indices_forwards_on_{}.json'.format(yesterday)
with open(file_path, 'w') as fp:
    json.dump(dictionnaire, fp)

In [ ]:
def cleaning_data_universe(path_to_processed_data,path_to_cleaned_data):
    print('################## Cleaning dividends universe ##################')
    
    data_cleaned = {}
    
    with open(path_to_processed_data) as json_file:
        data = json.load(json_file)
        
    for indice in list(data.keys()):
        if data[indice] is not None:
            if np.sum(np.isnan(data[indice]))==0:
                data_cleaned[indice] = data[indice]
        
    with open(path_to_cleaned_data, 'w') as fp:
        json.dump(data_cleaned, fp)
    print('file saved')

In [ ]:
path_to_processed_data = 'data/processed/UNIVERSE/Dividends_Forwards/universe_indices_forwards_on_{}.json'.format(yesterday)
path_to_cleaned_data = 'data/cleaned/UNIVERSE/Dividends_Forwards/universe_indices_forwards_on_{}.json'.format(yesterday)

cleaning_data_universe(path_to_processed_data,path_to_cleaned_data)

In [ ]:
with open(path_to_cleaned_data) as json_file:
    data = json.load(json_file)

In [ ]:
import scripts.sqlite_populate as ssp

In [ ]:
import project.data_preprocessing as dp

In [ ]:
from scripts import sqlite_populate

In [ ]:
file_path = 'data/processed/UNIVERSE/universe_indices.npy'
universe_indices = np.load(file_path)